In [1]:
!pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client markdown2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.7 MB/s eta 0:00:00


In [17]:
from google.colab import auth
from googleapiclient.discovery import build

# Authenticate
auth.authenticate_user()

# Use the built-in credentials for Google Docs API
creds, _ = google.auth.default()
service = build('docs', 'v1', credentials=creds)


In [49]:
from google.colab import auth
from googleapiclient.discovery import build
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google.auth.exceptions import RefreshError
from google_auth_oauthlib.flow import InstalledAppFlow
import google.auth

# Authenticate using Google Colab's built-in method
auth.authenticate_user()

# Define Google API scopes
SCOPES = ['https://www.googleapis.com/auth/documents', 'https://www.googleapis.com/auth/drive']

# Load default credentials (Colab provides automatic authentication for Google Drive)
creds, _ = google.auth.default()

# Build the Google Docs API client
service = build('docs', 'v1', credentials=creds)

print("Authentication successful! Google Docs API is ready to use.")


Authentication successful! Google Docs API is ready to use.


In [7]:
import os
print(os.listdir())  # This will list all files in the current directory



['.config', 'credentials.json', 'sample_data']


In [23]:
markdown_text = """# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---
Meeting recorded by: Sarah Chen
Duration: 45 minutes
"""


In [21]:
# Create a new Google Doc
doc = service.documents().create(body={"title": "Product Team Sync - Meeting Notes"}).execute()
document_id = doc["documentId"]

print(f"Created document with ID: {document_id}")


Created document with ID: 1RwyEsiG3jEbCEVE8wl2L_hlu0LWWLli2HwEFaZx8RQg


In [55]:
def format_google_docs_content(service, document_id, markdown_text):
    #  Geting  actual document length before deleting
    doc = service.documents().get(documentId=document_id).execute()
    doc_length = doc.get("body").get("content")[-1]["endIndex"]

    #  Clear the existing content safely
    if doc_length > 1:
        requests = [{
            "deleteContentRange": {
                "range": {"startIndex": 1, "endIndex": doc_length - 1}  # Use actual document length
            }
        }]
        service.documents().batchUpdate(documentId=document_id, body={"requests": requests}).execute()

    #  Insert new formatted content
    lines = markdown_text.split("\n")
    requests = []

    # Prepare text for insertion
    text_content = ""
    for line in lines:
        # Remove # from headings before inserting
        if line.startswith("# "):
            text_content += line[2:] + "\n"
        elif line.startswith("## "):
            text_content += line[3:] + "\n"
        elif line.startswith("### "):
            text_content += line[4:] + "\n"
        else:
            text_content += line + "\n"

    # Insert entire content at once to prevent indexing issues
    requests.append({
        "insertText": {
            "location": {"index": 1},
            "text": text_content
        }
    })

    # Apply changes to clear and insert content
    service.documents().batchUpdate(documentId=document_id, body={"requests": requests}).execute()

    # Get updated document structure to fix index issues
    doc = service.documents().get(documentId=document_id).execute()
    content = doc.get("body").get("content")

    # Apply Formatting (Headings, Checkboxes, Mentions, Footer)
    requests = []  # Reset requests for styling
    start_index = 1  # Start index for formatting
    checkbox_count = 0  # Track number of checkboxes

    for element in content:
        if "paragraph" in element and "elements" in element["paragraph"]:
            for elem in element["paragraph"]["elements"]:
                text = elem["textRun"]["content"] if "textRun" in elem else ""
                text_start = elem["startIndex"]
                text_end = elem["endIndex"]

                #  heading styles
                if text.startswith("Product Team Sync"):  # Heading 1
                    requests.append({
                        "updateParagraphStyle": {
                            "range": {"startIndex": text_start, "endIndex": text_end},
                            "paragraphStyle": {"namedStyleType": "HEADING_1"},
                            "fields": "*"
                        }
                    })
                elif text.strip() in ["Attendees", "Agenda", "Action Items", "Next Steps", "Notes"]:  # Heading 2
                    requests.append({
                        "updateParagraphStyle": {
                            "range": {"startIndex": text_start, "endIndex": text_end},
                            "paragraphStyle": {"namedStyleType": "HEADING_2"},
                            "fields": "*"
                        }
                    })
                elif text.strip() in ["1. Sprint Review", "2. Current Challenges", "3. Next Sprint Planning"]:  # Heading 3 inside Agenda
                    requests.append({
                        "updateParagraphStyle": {
                            "range": {"startIndex": text_start, "endIndex": text_end},
                            "paragraphStyle": {"namedStyleType": "HEADING_3"},
                            "fields": "*"
                        }
                    })

                # Convert checkboxes only for Sarah, Mike, Anna, and David
                if text.startswith("- [ ]") and checkbox_count < 4:
                    requests.append({
                        "createParagraphBullets": {
                            "range": {"startIndex": text_start, "endIndex": text_end},
                            "bulletPreset": "BULLET_CHECKBOX"
                        }
                    })


                    requests.append({
                        "updateTextStyle": {
                            "range": {"startIndex": text_start, "endIndex": text_end},
                            "textStyle": {"fontSize": {"magnitude": 12, "unit": "PT"}},
                            "fields": "fontSize"
                        }
                    })

                    checkbox_count += 1  # Increment checkbox count

                # Highlight @mentions in blue and bold
                if "@sarah" in text or "@mike" in text or "@anna" in text or "@david" in text:
                    requests.append({
                        "updateTextStyle": {
                            "range": {"startIndex": text_start, "endIndex": text_end},
                            "textStyle": {
                                "bold": True,
                                "foregroundColor": {"color": {"rgbColor": {"red": 0, "green": 0, "blue": 1}}}
                            },
                            "fields": "*"
                        }
                    })

                # Format Footer (Meeting recorded by, Duration) - Italicized + Grey
                if "Meeting recorded by" in text or "Duration" in text:
                    requests.append({
                        "updateTextStyle": {
                            "range": {"startIndex": text_start, "endIndex": text_end},
                            "textStyle": {
                                "italic": True,
                                "foregroundColor": {"color": {"rgbColor": {"red": 0.5, "green": 0.5, "blue": 0.5}}}
                            },
                            "fields": "*"
                        }
                    })

    # Send formatting requests after text is inserted
    service.documents().batchUpdate(documentId=document_id, body={"requests": requests}).execute()

    print(f" Formatted document available at: https://docs.google.com/document/d/{document_id}")

# Run the function to format content in Google Docs
format_google_docs_content(service, document_id, markdown_text)


 Formatted document available at: https://docs.google.com/document/d/1RwyEsiG3jEbCEVE8wl2L_hlu0LWWLli2HwEFaZx8RQg
